In [1]:
import wod_predictor
from wod_predictor.data_loader import DataLoader
from wod_predictor.splitter import DataSpitter
from wod_predictor.preprocessor import DataPreprocessor 
from wod_predictor.modeling import RandomForestModel

In [2]:
data_path = wod_predictor.__path__[0].replace("wod_predictor", "Data")
loader = DataLoader(root_path = data_path, objects= ['open_results','descriptions','benchmark_stats',  'athlete_info'])
data = loader.load()

/storage/ice1/0/9/qzheng75/WOD-prediction/wod_predictor/data_loader.py:56: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))
/storage/ice1/0/9/qzheng75/WOD-prediction/wod_predictor/data_loader.py:56: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))


In [3]:
splitter = DataSpitter(sample = 1000, test_ratio=  0.2, test_filter='23.*')
train_data, test_data = splitter.split(data)

In [4]:
preprocessing_config = {
    'open_results':
    {
        'scale_up': True,
        'scale_args': {
            "method": "general",
            "scaler_name": "StandardScaler"
        }
    },
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero',
        'drop_missing_threshold': 0.9,
        'scale_args': {
            "method": "general",
            "scaler_name": "StandardScaler"
        }
        # 'neighbors': 5,
    },
    'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessor.fit(data=train_data)


/storage/ice1/0/9/qzheng75/WOD-prediction/wod_predictor/feature_engineering_parts/helpers.py:310: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_modified[workout_name] = pd.to_numeric(
/storage/ice1/0/9/qzheng75/WOD-prediction/wod_predictor/feature_engineering_parts/helpers.py:269: UserWarning: Could not convert -- to time, returning as is. Error: invalid literal for int() with base 10: '--'
  warnings.warn(


benchmark_df

In [5]:
train_preprocessed = preprocessor.transform(data=train_data)

/storage/ice1/0/9/qzheng75/WOD-prediction/wod_predictor/feature_engineering_parts/helpers.py:310: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_modified[workout_name] = pd.to_numeric(
/storage/ice1/0/9/qzheng75/WOD-prediction/wod_predictor/feature_engineering_parts/helpers.py:269: UserWarning: Could not convert -- to time, returning as is. Error: invalid literal for int() with base 10: '--'
  warnings.warn(


Converted height to height in imperial units
Converted weight to weight in imperial units


In [6]:
test_preprocessed = preprocessor.transform(data=test_data)

Converted height to height in imperial units
Converted weight to weight in imperial units


/storage/ice1/0/9/qzheng75/WOD-prediction/wod_predictor/feature_engineering_parts/helpers.py:310: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_modified[workout_name] = pd.to_numeric(


In [7]:

config = {}
rf_modeler = RandomForestModel(config = config, n_estimators = 10)
rf_modeler.fit(X = train_preprocessed['X'], y = train_preprocessed['y'])
rf_modeler.predict(X = test_preprocessed['X'])
rf_modeler.show_results(y_test = test_preprocessed['y'], meta_data = test_preprocessed['meta_data'])

Mean Absolute Error: 0.56
Mean Absolute Percentage Error: 1.91


,y_test_mean,y_pred_mean,error_mean,error_percentage
23.1,185.180000,185.544634,7.943454,4.289586
23.2A,130.890000,132.516508,13.067828,9.983824
23.2B,207.060000,208.446894,18.015811,8.700768
23.3,24.262705,23.686266,4.135994,17.046712
